In [1]:
import rioxarray
import numpy as np
import cupy as cp
from multiprocessing import Pool

In [2]:
file = 'test/data/mds_sao_paulo_city_4000.tiff'

In [3]:
xds = rioxarray.open_rasterio(file)

In [4]:
xds.rio.width, xds.rio.height

(4000, 4000)

In [5]:
a0 = cp.arange(10)
a0

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
cp.random.shuffle(a0)
a0

array([1, 3, 9, 8, 7, 2, 4, 5, 0, 6])

In [7]:
np.maximum.accumulate(a0.get())

array([1, 3, 9, 9, 9, 9, 9, 9, 9, 9])

In [8]:
thetas = 32 # deve ser divisível por 4
phis = 8
observer_h = 1.2 ## TODO
delta_theta = 2*np.pi/thetas
resolution = xds.rio.resolution()
tangents = np.tan(np.pi/2 - np.arcsin(np.arange(1 - 1/phis/2, 1/phis/2, -1/phis)))
indices = np.indices(xds.rio.shape)

In [9]:
svf = np.ones(xds.rio.shape, dtype='int16')
# svf_x = np.zeros(xds.rio.shape, dtype='int16')
# svf_y = np.zeros(xds.rio.shape, dtype='int16')
# grid_max = np.zeros(xds.rio.shape)

In [10]:
def rotate_2d(mds, angle):
    shape = mds.shape
    
    y, x = np.indices(shape,  dtype='float32')
    x, y = x - shape[0]//2, y[::-1] - shape[0]//2

    xr = np.int32(np.cos(angle) * x - np.sin(angle) * y[::-1] + shape[0]//2)
    yr = np.int32(np.sin(angle) * x + np.cos(angle) * y[::-1] + shape[0]//2)
    

    xr = np.where(xr < 0, 0, xr)
    xr = np.where(xr > shape[0]-1, shape[0]-1, xr)
    yr = np.where(yr < 0, 0, yr)
    yr = np.where(yr > shape[0]-1, shape[0]-1, yr)

    return mds[yr, xr]

In [11]:
def calc_svf_angle(r, t):
    # print("Efetuando Projecao")
    projection = np.rot90(mds_r, k=r) * t + resolution[0] * indices[1]
    # print("Projecao Acumulada")
    projection_acc = np.maximum.accumulate(projection, axis=1)
    # print("comparando ...")
    sky_is_visible = np.less_equal(projection_acc, projection)
    # print("rotacionando de volta")
    return np.rot90(sky_is_visible, k=-r)

In [12]:
## VErsão refatorada
for i in np.linspace(0, np.pi/2, thetas//4, endpoint=False):

    mds_r = rotate_2d(xds.values[0], i)
    print(np.rad2deg(i))

    p_loop = np.array([[r, t] for r in range(4) for t in tangents])

    with Pool(12) as p:
        svf_part = p.starmap(calc_svf_angle, zip(p_loop[:, 0], p_loop[:, 1]))

    svf += rotate_2d(np.sum(np.array(svf_part), axis=0), -i)



0.0
11.25
22.5
33.75
45.0
56.25
67.5
78.75


In [13]:
xds.values = svf.reshape(1, 4000, 4000)
xds.rio.to_raster('tmp/result_raytracing.tif')